# Investigation of alternative to baseline-dependent averaging.

The idea is to partition the visibility into two parts dependent on the density of samples in each grid cell. If the grid cell has many samples in it then we use a coarse gridding such as nearest neighbour. Only if there are few samples in a grid cell do we have to be careful about gridding. In this notebook, we investigate the consequences of imaging in this way.

In [ ]:
%matplotlib inline

import os
import sys

from time import clock

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['agg.path.chunksize'] = 10000
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as const
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt
from matplotlib.pyplot import cm 

from arl.visibility.operations import create_visibility
from arl.skymodel.operations import create_skycomponent
from arl.image.operations import show_image, export_image_to_fits, qa_image
from arl.image.iterators import raster_iter
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration
from arl.fourier_transforms.ftprocessor import *

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

doplot = True

Set up imaging parameters

In [ ]:
params = {'npol': 1,
          'cellsize': 0.001,
          'spectral_mode': 'channel',
          'channelwidth': 5e7,
          'reffrequency': 1e8,
          'kernel':'calculated'}

Construct the SKA1-LOW core configuration

In [ ]:
lowcore = create_named_configuration('LOWBD2-CORE')

We create the visibility holding the vis, uvw, time, antenna1, antenna2, weight columns in a table. The actual visibility values are zero.

In [ ]:
times = numpy.arange(- numpy.pi / 4.0, + numpy.pi * 1.001 / 4.0, numpy.pi / 160)
frequency = numpy.array([1e8])

reffrequency = numpy.max(frequency)
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-45.0 * u.deg, frame='icrs', equinox=2000.0)
vt = create_visibility(lowcore, times, frequency, weight=1.0, phasecentre=phasecentre, params=params)

In [ ]:
params = {'npixel': 512,
          'npol': 1,
          'cellsize': 0.001,
          'spectral_mode': 'channel',
          'channelwidth': 5e7,
          'reffrequency': 1e8,}

model = create_image_from_visibility(vt, params=params)

Now calculate the density of samples in the Fourier plane

In [ ]:
vts, density, densitygrid = weight_visibility(vt, model, params)
print(density.shape)
plt.clf()
uvdist = numpy.sqrt(vts.u*vts.u+vts.v*vts.v)
plt.plot(uvdist, density[:,0,0], '.')
plt.xlabel('UV distance (m)')
plt.ylabel('Density (weight per cell)')
plt.show()

In [ ]:
plt.clf()
plt.imshow(numpy.log(1e-3+densitygrid[0,0,...]))
plt.show()

Now make the PSF for the full data set and the dense and sparse data

In [ ]:

max_density = numpy.max(densitygrid)

params['kernel']='2d'  

ts = clock()
full_psf, full_sumwt = invert_2d(vt, model, dopsf=True, params=params)
full_psf.data /= full_sumwt
qa_full = qa_image(full_psf)
criterion = 'medianabs'

t_full = clock() - ts

for padding in [1, 2, 4, 8]:
    thresholds = []
    criteria = []
    t_2ds = []
    t_boxs = []

    for threshold in max_density * numpy.linspace(0., 1.0, 10)[1:]:
        vt_dense  = vt.select_rows(numpy.where(density[:,0,0]>threshold))
        vt_sparse = vt.select_rows(numpy.where(density[:,0,0]<=threshold))

        ts = clock()
        params['kernel']='2d' 
        params['padding']=2
        sparse_psf, sparse_sumwt = invert_2d(vt_sparse, model, dopsf=True, params=params)
        if sparse_sumwt > 0.0:
            sparse_psf.data /= sparse_sumwt
        t_2d = clock() - ts

        ts = clock()
        params['kernel']='box'
        params['padding']=padding
        dense_psf, dense_sumwt = invert_2d(vt_dense, model, dopsf=True, params=params)
        if dense_sumwt > 0.0:
            dense_psf.data /= dense_sumwt
        t_box = clock() - ts


        error_psf = copy.deepcopy(full_psf)
        error_psf.data -= (vt_dense.nvis * dense_psf.data + vt_sparse.nvis * sparse_psf.data) \
            / (vt_dense.nvis + vt_sparse.nvis)
    
        qa = qa_image(error_psf)
        t_boxs.append(t_box)
        t_2ds.append(t_2d)
        criteria.append(qa.data[criterion])
        thresholds.append(threshold)

    plt.clf()
    plt.subplot(121)
    t_2ds = numpy.array(t_2ds)
    t_boxs = numpy.array(t_boxs)
    plt.plot(thresholds, criteria/qa_full.data[criterion])
    plt.xlabel('Threshold (density)')
    plt.ylabel('Fractional %s' % (criterion))
    plt.title('Padding = %d' % (padding))
    
    plt.subplot(122)
    plt.plot(thresholds, t_2ds, color='r', label='sparse via 2d')
    plt.plot(thresholds, t_boxs, color='b', label='dense via box')
    plt.plot(thresholds, t_boxs+t_2ds, color='g', label='total')    
    plt.xlabel('Threshold (density)')
    plt.ylabel('Clock time (s)')
    plt.axhline(t_full, color='y', label='2d')
    plt.legend(loc='best')
    plt.title('Padding = %d' % (padding))
    plt.show()



We can see that for large padding, the constant FFT time starts to dominate and the approach becomes counterproductive.